# Part I. Creating an Environment

Here, we will define an environment (a Markov Decision Process) with a finite number of states and actions. 

The states are defined by their indexes: S = 0, 1, ..., Ns

And the same for the actions:            A = 0, 1, ..., Na

The file `finite_env.py` contains an abstract class `FiniteEnv` which is used to implement the main features of an environment:

* The transition probabilites, represented by an array `P` of shape (Ns, Na, Ns) such that `P[s, a, s']` = Prob($S_{t+1}=s'| S_t = s, A_t = a$);
* A list of all possible states `env.states`;
* An array containing the actions available in each state `env.action_sets`;
* The current state of the environment `env.state`;
* The discount factor `gamma`; 
* A function `reset` that puts an environment in a default state and returns this state (example: `initial_state = env.reset()`);
* A function `step` that takes a step in the environment, it takes as input an action and returns the next state, the reward, a flag that indicates whether we reached a terminal state, and a dictionary with extra information.

Read carefully the definition of the class `FiniteEnv` in `finite_env.py`. 

**Remark**: Here we put the discount factor `gamma` as an attribute of the environment, for convenience. However, we can also consider it an attribute of the agent instead. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np 
from finite_env import FiniteEnv

We will now create a class that implements `FiniteEnv`. We need to implement all the abstract methods, define the state/action sets, the transition probabibities and the discount factor. 

The class `ToyEnv1` below does that, and represents a MDP with 3 states and 2 actions per state. A reward of 1 is obtained during the transition to the last state, and is 0 in all other cases.

In [2]:
class ToyEnv1(FiniteEnv):
    """
    Enviroment with 3 states and 2 actions per state that gives a reward of 1 when going to the
    last state and 0 otherwise.

    Args:
        gamma (float): discount factor
        seed    (int): Random number generator seed

    """

    def __init__(self, gamma=0.99, seed=42):
        # Set seed
        self.RS = np.random.RandomState(seed)

        # Transition probabilities
        # shape (Ns, Na, Ns)
        # P[s, a, s'] = Prob(S_{t+1}=s'| S_t = s, A_t = a)

        Ns = 3
        Na = 2
        P = np.zeros((Ns, Na, Ns))

        P[:, 0, :] = np.array([[0.25, 0.5, 0.25], [0.1, 0.7, 0.2], [0.1, 0.8, 0.1]])
        P[:, 1, :] = np.array([[0.3, 0.3, 0.4], [0.7, 0.2, 0.1], [0.25, 0.25, 0.5]])

        # Initialize base class
        states = np.arange(Ns).tolist()
        action_sets = [np.arange(Na).tolist()]*Ns
        super().__init__(states, action_sets, P, gamma)

    def reward_func(self, state, action, next_state):
        return 1.0 * (next_state == self.Ns - 1)

    def reset(self, s=0):
        self.state = s
        return self.state

    def step(self, action):
        next_state = self.sample_transition(self.state, action)
        reward = self.reward_func(self.state, action, next_state)
        done = False
        info = {}
        self.state = next_state

        observation = next_state
        return observation, reward, done, info

    def sample_transition(self, s, a):
        prob = self.P[s,a,:]
        s_ = self.RS.choice(self.states, p = prob)
        return s_

Now lets see how the environment works:

In [3]:
# Create ToyEnv1 object
env = ToyEnv1(gamma=0.95)

# Print some properties
print("ToyEnv1 has %d states and %d actions"%(env.Ns, env.Na))
print("The array representing the transition probabilities has shape ", env.P.shape)
# When an action is fixed, we obtain the transition kernel of a Markov chain:
print("\n Transition kernel for a = 0 \n", env.P[:, 0, :])
print("\n Transition kernel for a = 1 \n", env.P[:, 1, :])

ToyEnv1 has 3 states and 2 actions
The array representing the transition probabilities has shape  (3, 2, 3)

 Transition kernel for a = 0 
 [[0.25 0.5  0.25]
 [0.1  0.7  0.2 ]
 [0.1  0.8  0.1 ]]

 Transition kernel for a = 1 
 [[0.3  0.3  0.4 ]
 [0.7  0.2  0.1 ]
 [0.25 0.25 0.5 ]]


In [4]:
# Get initial state
state = env.reset()
print("Initial state = ", state)

# Choose an action
a = 0

# Sample a transition
next_state, reward, done, info = env.step(a)
assert next_state==env.state # check that environment has been indeed updated
print("Action = %d, Next state = %d, Reward = %0.2f"%(a, next_state, reward))

Initial state =  0
Action = 0, Next state = 1, Reward = 0.00


# Part II. Bellman operator

### Defining a policy 
A deterministic Markov policy can be represented by a 1d array such that `policy[state]=action` represents the action to be taken in each state.

In [5]:
# A random policy
policy = np.random.randint(env.Na, size = (env.Ns,))
print("random policy = ", policy)

random policy =  [0 0 0]


## Exercise 1: Bellman operator and greedy policies
1. Write a function that takes an environment, a policy and a value function $V$ as input and returns the Bellman operator applied to $V$, $T^\pi V$.
2. Write a function that takes an environment and a value function $V$ as input and returns the Bellman optimality operator applied to $V$, $T^* V$ and the greedy policy with respect to $V$.
3. Let $V_1$ and $V_2$ be value functions. Verify the contraction property: $||T^\pi V_1 - T^\pi V_2|| \leq \gamma ||V_1 - V_2||$ and $||T^* V_1 - T^* V_2|| \leq \gamma ||V_1 - V_2||$, where $||V|| = \max_s |V(s)|$.
4. Verify that: if $V_1 \leq V_2$, then $T^\pi V_1 \leq T^\pi V_2$ and $T^* V_1 \leq T^* V_2$.
5. Verify that: $T^\pi V \leq T^* V$


### Reminder

$$
T^\pi V(s) = \sum_{s'}P(s'|s,\pi(s))[ r(s, \pi(s), s') + \gamma V(s')] \quad \mbox{ for a deterministic policy } \pi 
$$

$$
T^* V(s) = \max_a \sum_{s'}P(s'|s,a)[ r(s, a, s') + \gamma V(s')]  
$$


In [6]:
# --------------
# Solution to 1.
# --------------
def bellman_operator(V, policy, env):
    """
    Bellman operator. To be done!
    """
    TV = np.zeros(env.Ns)

    for s in env.states:
        a = policy[s]
        assert a in env.available_actions(s) # make sure this action is available
        prob = env.P[s, a, :]
        rewards = np.array([env.reward_func(s,a, s_) for s_ in env.states])
        TV[s] = np.sum( prob*(rewards + env.gamma*V)  )
        
    return TV

In [7]:
# --------------
# Solution to 2.
# --------------
def bellman_opt_operator(V, env):
    """
    Bellman optimality operator. To be done!
    """
    Q = -np.inf*np.ones((env.Ns, env.Na))
    greedy_policy = np.zeros(env.Ns)
    for s in env.states:
        for a in env.available_actions(s):
            prob = env.P[s, a, :]
            rewards = np.array([float(env.reward_func(s,a, s_)) for s_ in env.states])
            Q[s,a] = np.sum( prob*(rewards + env.gamma*V)  )

    TV = np.max(Q, axis = 1)
    argmax = np.argmax(Q, axis = 1)
    greedy_policy = argmax
    
    return TV, greedy_policy

In [8]:
# --------------
# Solution to 3.
# --------------
n_simulations = 5

print("Contraction of Bellman operator: ")
for ii in range(n_simulations):
    V1 = np.random.randn(env.Ns)
    V2 = np.random.randn(env.Ns)
    policy = np.random.randint(env.Na, size = (env.Ns,))

    # Contraction of Bellman operator
    contraction = np.abs(bellman_operator(V1, policy, env) - bellman_operator(V2, policy, env)).max() / np.abs(V1-V2).max()
    print(contraction)
    assert contraction <= env.gamma

print("\n Contraction of Bellman optimality operator: ")
for ii in range(n_simulations):
    V1 = np.random.randn(env.Ns)
    V2 = np.random.randn(env.Ns)

    # Contraction of Bellman operator
    contraction_opt = np.abs(bellman_opt_operator(V1, env)[0] - bellman_opt_operator(V2, env)[0]).max() / np.abs(V1-V2).max()
    print(contraction_opt)
    assert contraction_opt <= env.gamma
    

Contraction of Bellman operator: 
0.39128243097740323
0.45359051706104503
0.4981154400181063
0.5701564639131153
0.2690373847817199

 Contraction of Bellman optimality operator: 
0.683260900422996
0.018013580143773047
0.37461943626749716
0.604173184855166
0.6257444428610531


In [9]:
# --------------
# Solution to 4.
# --------------

print("Diff of Bellman operator: ")
for ii in range(n_simulations):
    V1 = np.random.randn(env.Ns)
    V2 = V1 + np.random.randn(env.Ns).clip(min=0)
    policy = np.random.randint(env.Na, size = (env.Ns,))
    
    # Diff of Bellman operator
    diff = bellman_operator(V1, policy, env) - bellman_operator(V2, policy, env)
    print(diff)
    assert (diff > 0).sum() == 0

print("\n Diff of Bellman optimality operator: ")
for ii in range(n_simulations):
    V1 = np.random.randn(env.Ns)
    V2 = V1 + np.random.randn(env.Ns).clip(min=0)

    # Diff of Bellman operator
    diff_opt = bellman_opt_operator(V1, env)[0] - bellman_opt_operator(V2, env)[0]
    print(diff_opt)
    assert (diff_opt > 0).sum() == 0

Diff of Bellman operator: 
[-0.39799087 -0.26532724 -1.06130898]
[-0.83973658 -1.06009967 -0.47763406]
[0. 0. 0.]
[-0.13969527 -0.21406868 -0.14184157]
[0. 0. 0.]

 Diff of Bellman optimality operator: 
[-0.32476496 -0.45873184 -0.33044808]
[-1.09552885 -1.2590533  -1.58226833]
[-0.06974172 -0.02324724 -0.0581181 ]
[-0.19422855 -0.27191998 -0.31076569]
[-0.21883263 -0.08753305 -0.08753305]


In [10]:
# --------------
# Solution to 5.
# --------------

print("Diff between Bellman operator and Bellman optimality operator: ")
for ii in range(n_simulations):
    V1 = np.random.randn(env.Ns)
    policy = np.random.randint(env.Na, size = (env.Ns,))
    
    # Diff between Bellman operator and Bellman optimality operator
    diff = bellman_operator(V1, policy, env) - bellman_opt_operator(V1, env)[0]
    print(diff)
    assert (diff > 0).sum() == 0

Diff between Bellman operator and Bellman optimality operator: 
[0. 0. 0.]
[-0.39436934 -0.25200253 -1.06520198]
[-0.27041804 -0.44143221  0.        ]
[ 0.         -0.91058268  0.        ]
[-0.32449739  0.          0.        ]


# Part III. Dynamic Programming

When the transition probabilities and reward functions are known, we can use dynamic programming algorithms to find an optimal policy or the optimal value function, that is, "solve" the MDP. 

In this part, you will implement Value Iteration and Policy Iteration to solve an MDP.

## Exercise 2: Value Iteration 

1. (Policy evaluation) Write a function that takes as input an initial value function `V0`, a policy `pi` and an environment `env` and returns a vector `V` such that $||T^\pi V -  V ||_\infty \leq \varepsilon $.
2. (Optimal Value function) Write a function that takes as input an initial value function `V0` and an environment `env` and returns a vector `V` such that $||T^* V -  V ||_\infty \leq \varepsilon $ and the greedy policy with respect to $V$.
3. Test the convergence of the functions you implemented.


Reminder:

* For any $V_0$, let $V_n = T^\pi V_{n-1}$. We have $\lim_{n\to\infty}V_n = V^\pi$ and $V^\pi = T^\pi V^\pi$
* For any $V_0$, let $V_n = T^* V_{n-1}$. We have $\lim_{n\to\infty}V_n = V^*$ and $V^* = T^* V^*$

In [11]:
# --------------
# Solution to 1.
# --------------
def value_iteration(V0, pi, env, epsilon=1e-5):
    """
    Finding the value of a policy. To be done!
    """
    it = 1
    V = V0
    while True:
        TV = bellman_operator(V, pi, env)        
        err = np.abs(TV-V).max() 
        assert np.sum((TV - V) < 0.0) == 0.0, "V must increase!"

        if err < epsilon:
            return TV 

        V = TV
        it += 1

In [12]:
# --------------
# Solution to 2.
# --------------
def opt_value_iteration(V0, env, epsilon=1e-5):
    """
    Finding the optimal value function. To be done!
    """
    it = 1
    V = V0
    while True:
        TV, greedy_policy = bellman_opt_operator(V, env)

        err = np.abs(TV-V).max() 
        if err < epsilon:
            return TV, greedy_policy

        V = TV
        it += 1

In [13]:
# --------------
# Solution to 3.
# --------------
epsilon = 1e-6
V0 = np.zeros(env.Ns)
pi = np.random.randint(env.Na, size = (env.Ns,))

V_ = value_iteration(V0, pi, env, epsilon)
err = np.abs(V_ - bellman_operator(V_, pi, env)).max()
print("norm of T_pi(V) - V = ", err)
assert err <= epsilon


V_, greedy_policy = opt_value_iteration(V0, env, epsilon)
err = np.abs(V_ - bellman_opt_operator(V_, env)[0]).max()
print("norm of T^*(V) - V = ", err)
assert err <= epsilon


norm of T_pi(V) - V =  9.214146272640278e-07
norm of T^*(V) - V =  9.158036746725884e-07


## Exercise 3: Exact Policy Evaluation

The value function $V^\pi$ of a policy $\pi$ can also be solved exactly by a linear system: $(I - \gamma P_\pi) V^\pi = r_\pi $ where $P_\pi$ is the transition kernel (matrix) induced by $\pi$: $P_\pi(s, s') = P(S_{t+1}=s' | S_t = s, A_t = \pi(s)) $ and $r_\pi(s') = \sum_{s} r(s,\pi(s),s') P(S_{t+1}=s' | S_t = s, A_t = \pi(s)) $ is the average reward obtained in state $s'$ by following the policy $\pi$.  

1. Write a function `exact_policy_eval` that takes as input an environment `env` and a policy `pi` and returns the exact value of $V^\pi$.
2. Let:

    `V_opt, greedy_policy = opt_value_iteration(V0, env)`
    and 
    
    `V_pol = exact_policy_eval(greedy_policy, env)`
    
    Verify that `V_opt` and `V_pol` are close enough. 

In [14]:
# --------------
# Solution to 1.
# --------------
def exact_policy_eval(pi, env):
    """
    To be done!
    """
    # Compute the transition matrix P_pi and reward vector r_pi
    P_pi = np.zeros((env.Ns, env.Ns))
    r_pi = np.zeros(env.Ns)
    
    for s in env.states:
        for s_ in env.states:
            a = pi[s]
            P_pi[s, s_] = env.P[s,a,s_]
            r_pi[s] += env.reward_func(s,a,s_)*env.P[s,a,s_]

    A = np.eye(env.Ns) - env.gamma*P_pi
    b = r_pi

    # Solve linear system
    V = np.linalg.solve(A, b)
    
    return V

In [15]:
# --------------
# Solution to 2.
# --------------
epsilon = 1e-6
V0 = np.zeros(env.Ns)
pi = np.random.randint(env.Na, size = (env.Ns,))

V_opt, greedy_policy = opt_value_iteration(V0, env, epsilon)
V_pol = exact_policy_eval(greedy_policy, env)


err = np.abs(V_ - bellman_opt_operator(V_, env)[0]).max()
print("difference = ", err)
print("V_opt = ", V_opt)
print("V_pol = ", V_pol)

difference =  9.158036746725884e-07
V_opt =  [7.21271907 6.90579408 7.33932563]
V_pol =  [7.21273739 6.90581239 7.33934395]


## Exercise 4: Policy Iteration

1. Write a function `policy_iteration` that takes an initial policy `pi0` and an environment `env` as input and returns the optimal policy.
2. Let:

    `V_opt, greedy_policy = opt_value_iteration(V0, env)`
    and 
    
    `pi = policy_iteration(pi0, env)`
    
    Verify that `greedy_policy` and `pi` are equal. 
    
Reminder:

* For any $\pi_0$, let $\pi_k$ be the greedy policy with respect to $V^{\pi_{k-1}}$, that is:

$$
  \pi_k(s) \in \arg\max_a \sum_{s'}P(s'|s,a)[ r(s, a, s') + \gamma V^{\pi_{k-1}}(s')]
$$

Then, there exists a value $n^* < \infty$ such that $\pi_{n^*}$ is an optimal policy.

In [16]:
# --------------
# Solution to 1.
# --------------
def policy_iteration(pi0, env):
    """
    To be done!
    """
    it = 1
    policy = pi0
    while True:
        # Policy evaluation
        V = exact_policy_eval(policy, env)

        # Policy improvement
        TV, greedy_policy = bellman_opt_operator(V, env)
        if (np.abs(TV-V).max() < 1e-12):
            return greedy_policy

        policy = greedy_policy
        it += 1


In [17]:
# --------------
# Solution to 2.
# --------------
V0 = np.zeros(env.Ns)
pi0 = np.random.randint(env.Na, size = (env.Ns,))

V_opt, greedy_policy = opt_value_iteration(V0, env)
pi = policy_iteration(pi0, env)

print(pi)
print(greedy_policy)

[1 1 1]
[1 1 1]


# Part IV. Value Prediction Algorithms

When the transition dynamics and the reward function are *not* available, we cannot use dynamic programming algorithms to evaluate a policy or find a near-optimal policy. In this case, we must rely on samples of trajectories to estimate them.

In this part, you will implement algorithms to estimate the value of a policy using samples collected by an agent interacting with the environment.

##### Sampling a trajectory 

The algorithms in this part require the agent to interact with the environment to sample trajectories. The code below illustrates how to sample one trajectory.

In [18]:
# Sampling a trajectory given a policy
policy = [1, 0, 1]
state = env.reset() # initial state
T = 10  # length of the trajectory

trajectory = []
for t in range(T):
    action = policy[state]
    next_state, reward, done, info = env.step(a)
    if done:
        break # stop if we reach a terminal state
    trajectory += [(state, action, next_state, reward)]
    print(trajectory[t])
    state = next_state


(0, 1, 2, 1.0)
(2, 1, 1, 0.0)
(1, 0, 1, 0.0)
(1, 0, 1, 0.0)
(1, 0, 1, 0.0)
(1, 0, 0, 0.0)
(0, 1, 2, 1.0)
(2, 1, 1, 0.0)
(1, 0, 1, 0.0)
(1, 0, 0, 0.0)


## Exercise 5.1: TD(0) 

Given a trajectory $ (x_t, x_{t+1}, r_t)_{t\geq 0} $ , the $t$-th step of TD(0) performs the following calculations:

$ \delta_t = r_t + \gamma \hat{V}_t(x_{t+1}) - \hat{V}_t(x_t)$

$ \hat{V}_{t+1}(x) = \hat{V}_t(x) + \alpha_t(x)\delta_t\mathbb{1}\{x=x_t\}  $ 

where $\alpha_t(x_t)$ is the step size and $\delta_t$ is called *temporal difference*.

Write a function `td_zero` that takes as input an environment `env`, a time horizon `T`, a policy `pi` and returns $\hat{V}_T$ and `V_it`, a 2d array such that `V_it[t, :]` is equal to $\hat{V}_t$.

In [19]:
def td_zero(env, pi, T):
    """
    TD(0) algorithm. To be done!
    """
    V = np.zeros(env.Ns)
    V_it = np.zeros((env.Ns, T))
    return V, V_it


## Exercise 5.2: Every-visit Monte-Carlo

Given a trajectory $ (x_t, x_{t+1}, r_t)_{t\geq 0} $ and a time horizon $T$ , the $t$-th step of every-visit MC performs the following calculations:

$ R_t = \sum_{s=t}^T \gamma^{s-t}r_s $

$ \delta_t = R_t - \hat{V}_t(x_t)$

$ \hat{V}_{t+1}(x) = \hat{V}_t(x) + \alpha_t(x)\delta_t \mathbb{1}\{x=x_t\}  $ 

Write a function `every_visit_mc` that takes as input an environment `env`, a time horizon `T`, a policy `pi` and returns $\hat{V}_T$ and `V_it`, a 2d array such that `V_it[t, :]` is equal to $\hat{V}_t$.


In [20]:
def every_visit_mc(env, pi, T):
    """
    Every-visit Monte-Carlo. To be done!
    """
    V = np.zeros(env.Ns)
    V_it = np.zeros((env.Ns, T))
    return V, V_it

## Exercise 5.3 TD($\lambda$)

Given a trajectory $ (x_t, x_{t+1}, r_t)_{t\geq 0} $ and, the $t$-th step of TD($\lambda$) performs the following calculations:

$ \delta_t = r_t + \gamma \hat{V}_t(x_{t+1}) - \hat{V}_t(x_t)$

$ z_{t+1}(x) = \mathbb{1}\{x=x_t\} + \gamma \lambda z_t(x)  $ 

$ \hat{V}_{t+1}(x) = \hat{V}_t(x) + \alpha_t(x)\delta_t z_{t+1}(x)  $ 

$ z_0(x) = 0 $

for all states $x$.
 
Write a function `td_lambda` that takes as input an environment `env`, a time horizon `T`, a policy `pi` and returns $\hat{V}_T$ and `V_it`, a 2d array such that `V_it[t, :]` is equal to $\hat{V}_t$.


In [21]:
def td_lambda(env, pi, l, T):
    """
    TD(lambda) algorithm. To be done!
    """
    V = np.zeros(env.Ns)
    V_it = np.zeros((env.Ns, T))
    z = np.zeros(env.Ns)        
    return V, V_it

## Exercise 5.4: Comparing the algorithms 

Using the functions implemented above, compare the convergence of TD(0), every-visit Monte-Carlo and TD(lambda) for different values of lambda. You can use the dynamic programming functions that you implemented before to compare the true value function and its estimate $\hat{V}_t$.

In [22]:
# ---------------
# 5.4
# ---------------

## Exercise 5.5: Non-ergodic MDPs 

The value function $\hat{V}(x)$ estimated by the algorithms above converges to the true value function under some conditions on the step size and provided that all states are visited infinitely often, which is not always the case in a single trajectory. In an ergodic MDP (i.e., the Markov chain induced by any policy is ergodic), a single trajectory is enough. How would you modify the functions above to ensure the convergence to the true value function? (no need to implement the modifications).

In [23]:
# ---------------
# 5.5
# ---------------

# Answer

# Part V. Control Algorithms

In Part IV, you implemented algorithms to estimate the value of a policy. Now, you will implement algorithms to estimate the optimal value function.

## Exercise 6.1: Q-Learning

Q-learning is an algorithm to estimate the Q function, which represents the value of each state-action pair:

$$
Q(s,a) = r(s,a) + \gamma \sum_{s'} P(s'|s,a) \max_a Q(s',a)
$$

If we have access to the Q function, an optimal policy can be computed as:

$$ \pi^*(s) \in \arg\max_a Q(s,a) $$

The Q-Learning algorithm allows us to estimate the optimal Q function using only trajectories from the MDP obtained by following a $\varepsilon$-greedy policy. The Q-learning update at time $t$ is done as follows:

1. In state $s_t$, take action $a_t$  such that $a_t$ is random with probability $\varepsilon$ and $a_t \in \arg\max_a \hat{Q}_t(s_t,a) $ with probability $1-\varepsilon$;
2. Observe $s_{t+1}$ and reward $r_t$;
3. Compute $\delta_t = r_t + \gamma \max_a \hat{Q}_t(s_{t+1}, a) - \hat{Q}_t(s_t, a_t)$;
4. Update $\hat{Q}_{t+1}(s, a) = \hat{Q}_t(s, a) + \alpha_t(s,a)\delta_t\mathbb{1}\{s=s_t, a=a_t\}  $


Implement the Q-Learning algorithm and verify that it converges to the optimal value function.

In [24]:
# ---------------
# 6.1 (implementing Q-Learning)
# ---------------


In [25]:
# ---------------
# 6.1 (check the convergence of Q-Learning)
# ---------------


## Exercise 6.2: SARSA

We can also define the Q function for a given policy $\pi$ (possibly stochastic):

$$
Q^\pi(s,a) = r(s,a) + \gamma \sum_{s'} P(s'|s,a) \sum_{a'}Q^\pi(s',a') \pi(a'|s')
$$

SARSA is similar to Q-learning, but instead of following a $\varepsilon$-greedy policy, it follows an exploratory (stochastic) policy $\pi_Q$ and estimates the value of this policy (it is an *on-policy* algorithm). One possible choice is:

$$
\pi_Q(a|s) = \frac{ \exp(\tau^{-1}Q(s,a))  }{\sum_{a'}\exp(\tau^{-1}Q(s,a')) }
$$
where $\tau$ is a "temperature" parameter: when $\tau$ approaches 0, $\pi_Q(a|s)$ approaches the greedy (deterministic) policy $a \in \arg\max_{a'}Q(s,a')$.

At each time $t$, SARSA keeps an estimate $\hat{Q}_t$ of the true Q function and uses $\pi_{\hat{Q}_t}(a|s)$ to choose the action $a_t$. If $\tau \to 0$ with a proper rate as $t \to \infty$, $\hat{Q}_t$ converges to $Q$ and $\pi_{\hat{Q}_t}(a|s)$ converges to the optimal policy $\pi^*$. 

The SARSA update at time $t$ is done as follows:

1. In state $s_t$, take action $a_t$ sampled at the previous step using $\pi_{\hat{Q}_{t-1}}(a|s_t)$ ;
2. Observe $s_{t+1}$ and reward $r_t$;
3. Sample the next action $a_{t+1} \sim \pi_{\hat{Q}_t}(a|s_{t+1})$;
4. Compute $\delta_t = r_t + \gamma \hat{Q}_t(s_{t+1}, a_{t+1}) - \hat{Q}_t(s_t, a_t)$
5. Update $\hat{Q}_{t+1}(s, a) = \hat{Q}_t(s, a) + \alpha_t(s,a)\delta_t\mathbb{1}\{s=s_t, a=a_t\}$


Implement the SARSA algorithm and verify that it converges to the optimal value function.

In [26]:
# ---------------
# 6.2 (implementation of SARSA)
# ---------------

In [27]:
# ---------------
# 6.2 (convergence of SARSA)
# ---------------